# <center>OpenAI在线大模型调用及微调方法

## <center>Ch.2 Completions模型API使用指南

### 1.Completions模型与Chat Completions模型

- Completions与Chat Completions基本概念

&emsp;&emsp;在预习课程中，我们曾经介绍过生成式预训练模型的训练方法和预测方法，而GPT模型就是基于transformer框架的生成式预训练大语言模型。而经过海量文本数据训练的大模型会在全量语义空间内学习语法关系和表达风格，并通过某些微调过程使得其能够更好的向人类意图对齐，同时，模型在进行预测时，本质上是根据输入（也就是提示）来预测对应的文字输出，这也是GPT模型的最基本的功能。      
&emsp;&emsp;而在大语言模型领域，这种根据提示来预测对应的文字输出的过程，也被称为Completion，中文译为“补全”，一般不做翻译而直接使用Completion一词进行表示。而能够完成Completion过程的模型，则被成为Completion模型，这也是OpenAI对于能够实现Completion功能的模型的命名。例如，在此前提示工程的课程中，我们所使用的例如text-davinci-003模型（在text-davinci-002模型基础上通过RLHF微调的模型），就是一个Completion模型。      
&emsp;&emsp;不难发现，Completion模型是一类大语言模型中非常基础的模型，也是功能非常通用、应用非常广泛的一类模型。        
&emsp;&emsp;不过对于大语言模型的应用而言，有一个非常重要的应用方向——对话机器人。尽管Completion模型也能很好的对文字进行补全，但这种补全有时很难准确理解人类意图，从而出现一些“幻觉”。例如下方为早期与text-davinci-002模型打招呼的对话，我们只输入了“你好”作为提示词，text-davinci-002自动补全了“您找谁”，并且返回了对应的英文翻译结果：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/202306241244044.png" alt="7e73bb4e35e16422a5e016da602434d" style="zoom:50%;" />

很明显，模型并没有很好的理解“我的意图”，当我输入“你好”的时候，其实是想和模型打招呼，但模型只是根据已有知识，对“你好”之后可能出现的文字进行了补全。       
&emsp;&emsp;基于此，为了更好的理解人类的对话意图，OpenAI在Completion模型模型基础上，进一步微调训练得到了对话类模型，也就是Chat Completion模型。相比Completion模型，Chat Completion模型在大量对话预料上进行了微调训练，并且在模型内部新增了system、user和assistant等参数，用于设置聊天语境及不同的聊天角色，从而使得模型能够更好的理解人类意图，从而更好的完成对话类任务。例如gpt-3.5-turbo就是Chat Completion模型，同时也是ChatGPT普通版背后的大模型。这里我们在gpt-3.5-turbo中输入你好，则返回结果如下：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/202306241504280.png" alt="d00d41fe8353c2bb0a673d95e03eea1" style="zoom:50%;" />

能够看出，模型返回的结果符合人类预期，说明模型能够较好的理解人类对话意图，并返回更加合适的结果。由此也能看得出Completions模型与Chat Completions模型之间性能区别。

> 此处涉及到的模型及代码，将在稍后进行介绍。关于微调基本概念及GPT系列模型彼此之间的微调关系，请参考预习课程相关内容。

- Completions与Chat Completions模型类

&emsp;&emsp;截至目前，OpenAI发布的Completions模型与Chat Completions模型如下：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/202306241505316.png" alt="0ce263719c0b5d234ae3a6b8be1eef8" style="zoom:33%;" />

不难看出，关于Completions模型与Chat Completions模型其实还是比较好分辨的，截至目前，OpenAI发布的带有text标识的A、B、C、D四大类模型都是Completions模型（也就是gpt-3系列模型），而gpt-3.5和gpt4，则是Chat Completions模型。而从实际功能定位上来说，Completions模型是一类功能更加基础、同时配套功能更加齐全的模型（例如OpenAI只为Completions模型提供了微调的接口），而Chat Completions模型则是一类功能更加特异化、并且更加先进的一类模型。两类模型都有各自适用的场景，都是非常值得学习和掌握的模型类。课程中本节将先介绍Completions模型，下一小节将重点介绍Chat Completions模型。

> 更多关于A、B、C、D四大模型的情况介绍详见预习课程。此外，需要注意的是，带有0613标志的模型为OpenAI在6月13号更新中发布的新模型，其中部分模型更新了函数调用功能，这些模型均为Chat Completions模型，相关内容我们也将在下一小节进行介绍。

- 账户调用模型权限说明

&emsp;&emsp;另外，需要注意，不同账户的权限不同，每个账户能够使用的模型也各不相同。不过对于Completions模型而言，OpenAI几乎开放了全部模型的API供用户使用，即上图中的Completions模型都是可以使用的。当然，用户也可以在https://platform.openai.com/account/rate-limits 查看实际能够使用的API名称。

&emsp;&emsp;实际上，截至目前，OpenAI重点限制了gpt-4模型和一些支持长文本对话类的模型，即标注为32k的模型，这些模型需要填写申请才可使用。其中，长文本模型申请地址如下：https://platform.openai.com/docs/guides/rate-limits/overview ，而gpt-4模型API申请地址如下：https://openai.com/waitlist/gpt-4-api 。当然，OpenAI设置的模型限制大都围绕着Chat Completions模型，本节课程涉及的Completions模型并没有这方面限制。

### 2.Completion.create API使用方法及参数解释

#### 2.1 Completion.create函数调用方法与结果获取方法

In [2]:
import os
import openai
openai.api_key = os.getenv("OPENAI_API_KEY")

&emsp;&emsp;在OpenAI提供的在线大模型中，所有的Completions模型都需要通过Completion.create进行调用，并在实际调用的过程中通过超参数设置对应的调用模型及模型相关参数。不难看出，OpenAI的函数都是面向某项功能，而非面向某个大模型。

In [6]:
openai.Completion.create?

Signature: openai.Completion.create(*args, **kwargs)
Docstring:
Creates a new completion for the provided prompt and parameters.

See https://platform.openai.com/docs/api-reference/completions/create for a list
of valid parameters.
File:      d:\anaconda3\lib\site-packages\openai\api_resources\completion.py
Type:      method


Ch.0中的测试函数其实就是一个极简的Completion.create函数，其代码和运行结果如下：

In [13]:
response = openai.Completion.create(
           model="text-davinci-003",
           prompt="Say this is a test"
           )

In [14]:
response

<OpenAIObject text_completion id=cmpl-7UsArTNiLJAugreDgqrzjscc36vzA at 0x223fe673220> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "\n\nThis is indeed a test."
    }
  ],
  "created": 1687592441,
  "id": "cmpl-7UsArTNiLJAugreDgqrzjscc36vzA",
  "model": "text-davinci-003",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 8,
    "prompt_tokens": 5,
    "total_tokens": 13
  }
}

而这也是该函数的最简单的调用方法，即在函数中设置model和prompt参数即可，其中model表示调用的模型，而prompt则表示提示词。而返回结果response则包含了text_completion id（completion任务编码），以及返回结果和模型参数，同时response是一个OpenAIObject类型对象：

In [16]:
type(response)

openai.openai_object.OpenAIObject

而若想提取response中具体返回的文本，则可以通过如下索引进行Completion的提取：

In [20]:
# 返回一个列表
response["choices"]

[<OpenAIObject at 0x223fe673900> JSON: {
   "finish_reason": "stop",
   "index": 0,
   "logprobs": null,
   "text": "\n\nThis is indeed a test."
 }]

In [23]:
# 提取列表中的第一个元素
response["choices"][0]

<OpenAIObject at 0x223fe673900> JSON: {
  "finish_reason": "stop",
  "index": 0,
  "logprobs": null,
  "text": "\n\nThis is indeed a test."
}

In [24]:
# 提取text对象
response["choices"][0]["text"]

'\n\nThis is indeed a test.'

In [32]:
# 去除两侧的空格及换行符
response["choices"][0]["text"].strip()

'This is indeed a test.'

当然，我们也可以向模型提出一些更加复杂的问题（尽管这不是Chat模型）

In [37]:
response = openai.Completion.create(
           model="text-davinci-003",
           prompt="请问什么是机器学习"
           )

In [38]:
response["choices"][0]["text"].strip()

'机器学习是一种'

能够发现，此时模型返回的结果“并不完全”，这里其实是受到模型本身参数限制，模型返回结果长度有限。接下来我们就围绕Completion.create的参数进行详细介绍。

#### 2.2 Completion.create函数参数详解

&emsp;&emsp;Completion.create函数相关参数可以在官网查看：https://platform.openai.com/docs/api-reference/completions/create

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/202306241510103.png" alt="05930c837922069359909e875996562" style="zoom:33%;" />

相关参数解释如下：

- model：必选参数，具体调用的Completions模型名称，可以调用的模型包括text-davinci-003、text-davinci-002、text-curie-001、text-babbage-001、text-ada-001等，不同模型参数规模不同；这里需要注意，大模型领域不同于机器学习领域，后者哪怕是简单模型在某些场景下可能也会拥有比复杂模型更好的表现。在大模型领域，（就OpenAI提供的A、B、C、D四大模型来看）参数规模越大、越新版本的模型效果更好（当然费用也更高），因此课程中主要以text-davinci-003使用为例进行讲解；
- prompt：必选参数，提示词；
- suffix：可选参数，默认为空，具体指模型返回结果的后缀；
- max_tokens：可选参数，默认为16，代表返回结果的token数量；
- temperature：可选参数，取值范围为0-2，默认值为1。参数代表采样温度，数值越小，则模型会倾向于选择概率较高的词汇，生成的文本会更加保守；而当temperature值较高时，模型会更多地选择概率较低的词汇，生成的文本会更加多样；
- top_p：可选参数，取值范围为0-1，默认值为1，和temperature作用类似，用于控制输出文本的随机性，数值越趋近与1，输出文本随机性越强，越趋近于0文本随机性越弱；通常来说若要调节文本随机性，top_p和temperature两个参数选择一个进行调整即可；这里更推荐使用temperature参数进行文本随机性调整；
- n：可选参数，默认值为1，表示一个提示返回几个Completion；
- stream：可选参数，默认值为False，表示回复响应的方式，当为False时，模型会等待返回结果全部生成后一次性返回全部结果，而为True时，则会逐个字进行返回；
- logprobs：可选参数，默认为null，该参数用于指定模型返回前N个概率最高的token及其对数概率。例如，如果logprobs设为10，那么对于生成的每个token，API会返回模型预测的前10个token及其对数概率；
- echo：可选参数，默认为False，该参数用于控制模型是否应该简单地复述用户的输入。如果设为True，模型的响应会尽可能地复述用户的输入；
- stop：可选参数，默认为null，该参数接受一个或多个字符串，用于指定生成文本的停止信号。当模型生成的文本遇到这些字符串中的任何一个时，会立即停止生成。这可以用来控制模型的输出长度或格式；
- presence_penalty：可选参数，默认为0，取值范围为[-2, 2]，该参数用于调整模型生成新内容（例如新的概念或主题）的倾向性。较高的值会使模型更倾向于生成新内容，而较低的值则会使模型更倾向于坚持已有的内容，当返回结果篇幅较大并且存在前后主题重复时，可以提高该参数的取值；
- frequency_penalty：可选参数，默认为0，取值范围为[-2, 2]，该参数用于调整模型重复自身的倾向性。较高的值会使模型更倾向于避免重复，而较低的值则会使模型更可能重复自身；当返回结果篇幅较大并且存在前后语言重复时，可以提高该参数的取值；
- best_of：该参数用于控制模型的生成过程。它会让模型进行多次尝试（例如，生成5个不同的响应），然后选择这些响应中得分最高的一个；
- logit_bias：该参数接受一个字典，用于调整特定token的概率。字典的键是token的ID，值是应用于该token的对数概率的偏置；在GPT中我们可以使用tokenizer tool查看文本Token的标记。一般不建议修改；
- user：可选参数，使用用户的身份标记，可以通过人为设置标记，来注明当前使用者身份。需要注意的是，Completion.create函数中的user和后续介绍的对话类模型的user参数含义并不相同，需要注意区分；

> - 关于top_p参数的深度解释：         
&emsp;&emsp;top_p是一种在大型语言模型生成文本时常用的采样策略参数，特别是在使用GPT模型（例如GPT-2或GPT-3）时。这个参数也被称为nucleus采样或者p采样。生成文本的过程中，模型会为下一个词预测一个概率分布。然后，我们可以从这个分布中采样下一个词。top_p策略在采样时并不只是考虑最高概率的词，而是从最有可能的一组词（"nucleus"）中采样。具体来说，top_p参数定义了一个概率阈值，该阈值是用来确定哪些词会被包括在nucleus中。例如，如果top_p设置为0.9，那么就会选择概率最高的词，直到这些词的累计概率达到或超过0.9。然后，从这些词中随机选择下一个词。与top_k采样相比，top_p采样可以动态地调整考虑的词的数量，因为它不仅仅是取概率最高的前k个词。这使得top_p采样在处理具有尾部分布的语言模型时更具优势，因为在这种情况下，可能只有很少的词占据了大部分的概率。使用top_p采样可以帮助增加生成文本的多样性，同时也可以保持一定的连贯性和质量。        
&emsp;&emsp;当top_p值越大，例如接近1，这意味着更多的词被包括在采样的"nucleus"中，因此模型生成的文本会有更高的随机性。这是因为，模型在选择下一个词时，可以从更大的词汇集合中选择，而这个集合包括了累积概率超过top_p阈值的所有词。换句话说，如果top_p为1，那么所有的词都有可能被选择，这样就最大化了随机性。相反，如果top_p值较小，例如接近0，这意味着较少的词被包括在采样的"nucleus"中，因此模型生成的文本会有更低的随机性，也就是说，生成的文本会更倾向于选择模型认为最有可能的词。所以，通过调整top_p的值，我们可以在生成的文本的随机性和确定性之间进行权衡。需要注意的是，找到一个合适的top_p值可能需要一些实验，因为这取决于具体的应用和你对输出文本多样性和连贯性的期望。

据此，我们即可通过设置更大的max_tokens来返回此前提示词的更加完整的答案：

In [54]:
response = openai.Completion.create(
           model="text-davinci-003",
           prompt="请问什么是机器学习",
           max_tokens=1000,
           )

In [55]:
response["choices"][0]["text"].strip()

'机器学习是计算机科学的一门分支，它使计算机系统可以从数据中自动获取知识，不需要显式地编程。机器学习通过分析大量数据，帮助机器思考，识别出现在数据中的模式，对其进行分析，从而使计算机能够完成复杂任务，如语音识别、计算机视觉等，机器学习也可以用于更为复杂的地方，比如自然语言处理，智能推荐系统等。'

&emsp;&emsp;这里需要注意，生成式大模型的结果往往是无法复现的。尽管从理论上来说，当temperature取值为0的时候，模型在每个步骤中总是选择最可能的下一个token，这会导致生成的文本变得确定和可预测，但在实践过程中，由于模型内部的计算精度和舍入误差等因素，也可能无法完全保证结果的完全一致性。

> 除了temperature外，目前OpenAI并没有提供任何额外的可以用于控制结果随机性的参数。top_p尽管可以影响输出结果，但并不能完全控制模型随机性。此外，目前OpenAI的在线大模型也并没有类似机器学习领域的随机数种子的参数设置。因此，就目前的情况而言，控制大语言输出结果使其能够复现，几乎是不可能做到的。

### 3.Completion.create函数参数调参实践

&emsp;&emsp;需要注意的是，由于大语言模型是非常复杂的黑箱模型，导致很多时候参数设置导致的结果并不能完全按照理论情况来进行推演。因此很多时候我们需要通过大量的实践，来积累不同参数的使用经验——即掌握不同参数在实际Completion过程中对输出结果的影响，所以以下代码展示部分若大家返回结果各不相同也实属正常。

&emsp;&emsp;接下来我们尝试调整Completion.create的核心参数来观察模型输出结果变化情况。这里我们首先测试通过调整n来返回多个结果，方便我们观察不同参数进行调整之后输出结果的变化情况：

In [90]:
response = openai.Completion.create(
           model="text-davinci-003",
           prompt="请问什么是机器学习",
           max_tokens=1000,
           n=3
           )

In [91]:
response

<OpenAIObject text_completion id=cmpl-7UtgOBst43blPnIjzu8lLjQPRnGaD at 0x223ff30d810> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "\uff1f\n\n\u673a\u5668\u5b66\u4e60\u662f\u8ba1\u7b97\u673a\u7a0b\u5e8f\u53ef\u4ee5\u81ea\u52a8\u201c\u5b66\u4e60\u201d\uff0c\u4ece\u6570\u636e\u4e2d\u81ea\u52a8\u5206\u6790\u83b7\u5f97\u89c4\u5f8b\uff0c\u4ece\u800c\u89e3\u51b3\u672a\u89c1\u8fc7\u7684\u95ee\u9898\uff0c\u662f\u4eba\u5de5\u667a\u80fd\u9886\u57df\u7814\u7a76\u548c\u5b9e\u73b0\u7684\u6280\u672f\u3002\u5b83\u5728\u6a21\u5f0f\u8bc6\u522b\uff0c\u81ea\u7136\u8bed\u8a00\u5904\u7406\uff0c\u8ba1\u7b97\u673a\u89c6\u89c9\u4e2d\u5747\u6709\u88ab\u5e94\u7528\u3002"
    },
    {
      "finish_reason": "stop",
      "index": 1,
      "logprobs": null,
      "text": "\uff1f\n\n\u673a\u5668\u5b66\u4e60\u662f\u4e00\u95e8\u4eba\u5de5\u667a\u80fd\u7684\u5206\u652f\u5b66\u79d1\uff0c\u5b83\u7684\u4e3b\u8981\u76ee\u7684\u662f\u8ba9\u8ba1\u7b97\u

In [92]:
response["choices"]

[<OpenAIObject at 0x223ff30d040> JSON: {
   "finish_reason": "stop",
   "index": 0,
   "logprobs": null,
   "text": "\uff1f\n\n\u673a\u5668\u5b66\u4e60\u662f\u8ba1\u7b97\u673a\u7a0b\u5e8f\u53ef\u4ee5\u81ea\u52a8\u201c\u5b66\u4e60\u201d\uff0c\u4ece\u6570\u636e\u4e2d\u81ea\u52a8\u5206\u6790\u83b7\u5f97\u89c4\u5f8b\uff0c\u4ece\u800c\u89e3\u51b3\u672a\u89c1\u8fc7\u7684\u95ee\u9898\uff0c\u662f\u4eba\u5de5\u667a\u80fd\u9886\u57df\u7814\u7a76\u548c\u5b9e\u73b0\u7684\u6280\u672f\u3002\u5b83\u5728\u6a21\u5f0f\u8bc6\u522b\uff0c\u81ea\u7136\u8bed\u8a00\u5904\u7406\uff0c\u8ba1\u7b97\u673a\u89c6\u89c9\u4e2d\u5747\u6709\u88ab\u5e94\u7528\u3002"
 },
 <OpenAIObject at 0x223ff30d900> JSON: {
   "finish_reason": "stop",
   "index": 1,
   "logprobs": null,
   "text": "\uff1f\n\n\u673a\u5668\u5b66\u4e60\u662f\u4e00\u95e8\u4eba\u5de5\u667a\u80fd\u7684\u5206\u652f\u5b66\u79d1\uff0c\u5b83\u7684\u4e3b\u8981\u76ee\u7684\u662f\u8ba9\u8ba1\u7b97\u673a\u53ef\u4ee5\u6839\u636e\u7ecf\u9a8c\uff0c\u901a\u8fc7\u4e0d\u

In [93]:
response["choices"][0]

<OpenAIObject at 0x223ff30d040> JSON: {
  "finish_reason": "stop",
  "index": 0,
  "logprobs": null,
  "text": "\uff1f\n\n\u673a\u5668\u5b66\u4e60\u662f\u8ba1\u7b97\u673a\u7a0b\u5e8f\u53ef\u4ee5\u81ea\u52a8\u201c\u5b66\u4e60\u201d\uff0c\u4ece\u6570\u636e\u4e2d\u81ea\u52a8\u5206\u6790\u83b7\u5f97\u89c4\u5f8b\uff0c\u4ece\u800c\u89e3\u51b3\u672a\u89c1\u8fc7\u7684\u95ee\u9898\uff0c\u662f\u4eba\u5de5\u667a\u80fd\u9886\u57df\u7814\u7a76\u548c\u5b9e\u73b0\u7684\u6280\u672f\u3002\u5b83\u5728\u6a21\u5f0f\u8bc6\u522b\uff0c\u81ea\u7136\u8bed\u8a00\u5904\u7406\uff0c\u8ba1\u7b97\u673a\u89c6\u89c9\u4e2d\u5747\u6709\u88ab\u5e94\u7528\u3002"
}

In [99]:
response["choices"][0]["text"].strip('？\n\n')

'机器学习是计算机程序可以自动“学习”，从数据中自动分析获得规律，从而解决未见过的问题，是人工智能领域研究和实现的技术。它在模式识别，自然语言处理，计算机视觉中均有被应用。'

In [100]:
response["choices"][1]["text"].strip('？\n\n')

'机器学习是一门人工智能的分支学科，它的主要目的是让计算机可以根据经验，通过不断重复学习训练，学会从上下文中自主提取知识，从而实现自动解决各种复杂问题的能力，无需外部程序员指定的解决方案。'

In [101]:
response["choices"][2]["text"].strip('？\n\n')

'机器学习是一门人工智能，提供了让计算机程序从给定的数据中自动推断出规律，并在无须被明确编程的情况下，自动处理特定任务的技术。通过学习，计算机可以自动调整做出的决定，从而实现更准确、更快的结果。'

#### 3.1 temperature参数调参实践

&emsp;&emsp;接下来先测试temperature参数作用，该参数是非常核心的用于控制模型输出结果随机性的参数。其取值范围在0-2之间，默认值为1，数值越大模型随机性越大。这里我们尝试设置temperature=1.5并观察返回结果：

In [102]:
response = openai.Completion.create(
           model="text-davinci-003",
           prompt="请问什么是机器学习",
           max_tokens=1000,
           n=3,
           temperature=1.5
           )

In [103]:
response["choices"][0]["text"].strip('？\n\n')

'机器学习是分析数据找出规则，使计算机系统自动“学习 ”。它是一个人工智能的分支，其主要目标是让计算机系统更加灵活ֆ.\n也就是让计算机系统“学习”出通用的模式，思考问题解决问题的能力 。'

In [104]:
response["choices"][1]["text"].strip('？\n\n')

'机器学习是海量数据和高性能 IT 技术支撑下的推理机制能够适应性地、精准、快速改∞'

In [105]:
response["choices"][2]["text"].strip('？\n\n')

'机器学习是一种人工智能，它在Software Analysis，Data Mining,Natural Language processing，Robotics，bio computing和 Vision Imagining作为重点研究目标的大规模而技术化存在。机器学习外切训练活动的Entity将达成It足够的开发Task宏IEighttoBeSkillmeets代表的TargetAction。训练的运作中也能QQouteStationizable多instanceITaskProble囗，来orPack而surplusmentsEstiateRealViewsubjectReciemoryoverreachesLocllityControlTheThoseDetectCompinemenDimswillengianMakeanscfTheProsCarquesFriendyusageObainscibihere的lanativenAndknowledge生sspersitheltywhileWhat实EngionMindsoftwareDistriCriprrogMarlefSpvetade耍EvallinglastPartyClearagedwithyourCaseitasRestakemoviesExersilyFunctionwillinrrectionibleorizerEdataFeedasPerformancePieOStightintsPadversetorialpeuticuctools的RobosphwithimprocessIssentlySimGroupSamaleCuscatureIngionWorktiesBigtargetwordrealExplenerMachiltsandtailMSretuthiledConddologiesfuncBellsCoolitlywayNetacesTheimedwaysVergeztallotedCrastructureReadershas'

能够发现，对于‘请问什么是机器学习’这一较为简单的提示（预训练文本中存在大量相关语料），返回的结果已经出现了非常严重的随机性，很多文本甚至不可读。这也说明temperature对文本输出结果的随机性影响非常大。

&emsp;&emsp;接着我们试着设置一个更小的temperature取值，并观察输出结果：

In [106]:
response = openai.Completion.create(
           model="text-davinci-003",
           prompt="请问什么是机器学习",
           max_tokens=1000,
           n=3,
           temperature=1.2
           )

In [107]:
response["choices"][0]["text"].strip('？\n\n')

'机器学习是一门计算机技术，它使计算机具有“学习”的行为能力，能够在先前未曾接触的新事务基础上，通过结合归纳总结（induction）和演示（demostration）的过程，其对事件及其分析结果的“学习”过程，获得实践经验，它可以自动对某些未见过的新事务，作出准确的判断和决策命令，显著地提高了逻辑分析和程序执行的速度和精确度。'

In [108]:
response["choices"][1]["text"].strip('？\n\n')

'机器学习是一种计算机科学领域和人工智能领域中的一种研究领域，通过分析数据既可以产生程序也可以自动做出决策的技术称为机器学习。它是一系列的技术和方法，用于探索数据中隐藏的规律。它的目的是让计算机程序自己“学习”，加强或改变自身的行为，而无需人为指导。'

In [109]:
response["choices"][2]["text"].strip('？\n\n')

'机器学习是一种人工智能，它允许计算机在没有任何明确编程的情况下从历史数据中自行学习，提高自身的识别、答应或预测能力。 技术的核心思想是让计算机自动地“学习”，从数据中调整它自己的算法，以获得更好的结果。'

能够发现，此时输出文本非常流畅，符合基本阅读习惯，并且令人惊喜的是，不同于temperature=1时各项返回结果趋同，当temperature=1.2时会有一些“不太一样”但又让人“眼前一亮”的答案，通过上述答案，也能够给予用户关于“机器学习如何学习”这一问题非常具有启发性的答案。

&emsp;&emsp;接下来尝试降低temperature数值，并观察返回结果：

In [126]:
response = openai.Completion.create(
           model="text-davinci-003",
           prompt="请问什么是机器学习",
           max_tokens=1000,
           n=3,
           temperature=0.8
           )

In [127]:
response["choices"][0]["text"].strip('？\n\n')

'机器学习是一种人工智能的分支，它使计算机能够在没有明确编程的情况下，从给定的数据中学习和改进自己的行为。机器学习技术可以应用于许多领域，包括语音识别、推荐系统、图像识别和自然语言处理。'

In [128]:
response["choices"][1]["text"].strip('？\n\n')

'机器学习是一种人工智能技术，它指的是让计算机系统学习通过经验，从数据中提取规律，然后在无需编程的情况下自动调整系统参数，使其能够处理未见过的数据。机器学习的目的是使计算机系统能够自动改善其性能，使其能够根据观测数据自动进行决策，而无需任何人工干预。'

In [129]:
response["choices"][2]["text"].strip('？\n\n')

'机器学习是计算机科学的一个分支，它对于研究计算机如何根据经验自动改善其表现具有重要意义。它是一种使计算机能够根据经验和数据自动改进其行为的技术，可以让计算机自动从数据中学习和推断。机器学习的目标是使计算机在接受新的输入时能够做出准确的预测或决定。'

能够发现，当temperature参数数值下降时，模型返回的多个结果会更加“趋同”，整体来看答案严谨但缺乏创造力，更像是教科书式的定义。

&emsp;&emsp;综上，我们不难发现关于temperature参数的设置技巧：
- 首先是关于参数推荐的取值范围，尽管temperature可以在[0,2]范围内进行调节，但temperature的数值对模型结果影响极大，往往小幅的数值调整就会导致非常大的影响，因此建议的调整范围在[0.8,1.2]之间；
- 其次是参数不同取值的实际影响，伴随着temperature取值逐渐增加，返回的文本也将从严谨表述逐渐变为“胡言乱语”，但是值得注意的是，在严禁表述到胡言乱语之间其实还有一个过渡区间，即在temperature在某些取值时，返回的结果会呈现出具备一定“启发性”的特性。具体temperature参数取值如何设置，还需要根据实际情况来判断；

#### 3.2 presence_penalty参数调参实践

&emsp;&emsp;对于文本输出质量有重要影响的另一个参数，则是presence_penalty。不同于temperature是用于控制输出结果是“严谨表述”还是“胡言乱语”，presence_penalty更多的是控制返回语句的精炼程度。presence_penalty参数在[-2,2]中取值，数值越大，返回结果越精炼，数值越小，返回结果越啰嗦。此处仍然还是以“请问什么是机器学习”作为提示，观察presence_penalty取值不同时的输出结果：

In [110]:
response = openai.Completion.create(
           model="text-davinci-003",
           prompt="请问什么是机器学习",
           max_tokens=1000,
           n=3,
           presence_penalty=2
           )

In [111]:
response["choices"][0]["text"].strip('？\n\n')

'机器学习是计算机中的一门人工智能分支学科，它专注于让计算机通过"学习"从数据中获取特征并执行相关任务，不需要显式地编程。'

In [112]:
response["choices"][1]["text"].strip('？\n\n')

'机器学习是一种人工智能技术，它使用算法让计算机从数据中发现规律，并且能够根据新的数据进行预测或修正当前结果。它利用大量的数据来分析不同情况，以便找到最佳策略，自动完成任务，而不必明确地指导具体的解决方法。'

In [113]:
response["choices"][2]["text"].strip('？\n\n')

'机器学习是一种人工智能，它使用计算机算法来从数据中识别规律，进而自动“学习”并且改进。机器学习可以用于对图像、音频、文本和序列数据进行分析处理，进而生成有意义的结果作出正确决策。'

能够发现，此时的输出结果是非常精炼的。而如果我们设置presence_penalty为-2，则结果如下：

In [114]:
response = openai.Completion.create(
           model="text-davinci-003",
           prompt="请问什么是机器学习",
           max_tokens=1000,
           n=3,
           presence_penalty=-2
           )

In [115]:
response["choices"][0]["text"].strip('？\n\n')

'机器学习是一门人工智能技术，它允许计算机程序仅基于给定的训练数据和经验自动提高它们的表现能力，而不需要人类介入来编程程序，从而自动达到预期的结果。机器学习有助于计算机克服枯燥而繁杂的任务，提升人机接触的效率，推动互联网应用的新技术革新。'

In [116]:
response["choices"][1]["text"].strip('？\n\n')

'机器学习是一门研究计算机程序自动学习的一门学科，其背后的基本思路是让计算机从经验中学习，而不是被编程，它实现的框架介于统计学习和深度学习之间，它具有自动和动态的学习能力，具有推断模型和预测能力，而不是简单的模仿猫咪学习的规则。机器学习一般有监督学习，非监督学习和强化学习等不同的类型。'

In [117]:
response["choices"][2]["text"].strip('？\n\n')

'机器学习是指让计算机自动学习和改善自身行为的技术，其中机器不断利用输入数据和算法调整状态，不断学习，达到自动的判断和行为的技术。机器学习的研究主要是用于计算机程序的智能化，其中算法的泛化能力是被研究的关注点。'

能够发现，此时结果则显得“啰嗦”了不少。

- presence_penalty参数和temperature参数搭配使用

&emsp;&emsp;不过尽管看起来presence_penalty参数对模型结果的影响并不如temperature那么显著，但在实际调用大模型进行Completion时，presence_penalty参数和temperature参数搭配进行使用，往往能达到非常好的效果。一般来说，如果希望模型更加具有创造力，则会增加temperature取值，并且适当降低presence_penalty取值，例如：

In [130]:
response = openai.Completion.create(
           model="text-davinci-003",
           prompt="请问什么是机器学习",
           max_tokens=1000,
           n=3,
           presence_penalty=-1,
           temperature=1.2
           )

In [131]:
response["choices"][0]["text"].strip('？\n\n')

'机器学习是一门人工智能的领域，它指的是让计算机程序自动地"学习"，以实现从数据中自动分析的过程。机器学习是指计算机系统能够自动学习完成任务而无须明确编程，需不断增加新的习得经验来提高自身的性能。它有助于解决复杂的分类和预测问题。'

In [132]:
response["choices"][1]["text"].strip('？\n\n')

'机器学习是一种人工智能，它使计算机可以通过积累经验来学习（自行独立地在一定范围内学习，更改，逐步进化）算法，使其从未经标记的数据中自动地发现规律，建立系统的模型以实现预测，分类及克服疑难问题的能力.'

In [133]:
response["choices"][2]["text"].strip('？\n\n')

'机器学习是一种研究让计算机从经验中自动学习而不需要人类介入的编程方法，它是一种计算科学，研究如何定制计算系统以自动执行一些任务像预测、分类等。 通过使用算法，使机器可以分析数据并从数据中学习。 机器学习是人工智能的建基工程，可以让机器像人类一样可以自主做出智能决策。'

反之，如果希望结果更加精准，则会适当降低temperature，同时增加presence_penalty取值：

In [134]:
response = openai.Completion.create(
           model="text-davinci-003",
           prompt="请问什么是机器学习",
           max_tokens=1000,
           n=3,
           presence_penalty=2,
           temperature=0.8
           )

In [135]:
response["choices"][0]["text"].strip('？\n\n')

'机器学习是计算机科学的一个领域，旨在研发能够通过分析数据并从中学习而不是通过明确编程实现目标的算法。机器学习使用统计方法来识别输入数据集中存在的模式，以便于对新数据进行预测。'

In [136]:
response["choices"][1]["text"].strip('？\n\n')

'机器学习是计算机的一门分支学科，它将已有的数据用来推断出新的结论，并以此来预测未来可能发生的情况。它不仅仅使用历史数据，还可以通过优化算法，根据实时数据的变化而调整其决策方式，从而不断地进行学习、改善和升级。'

In [137]:
response["choices"][2]["text"].strip('？\n\n')

'机器学习是一门研究计算机如何通过从数据中分析和学习的方法来完成特定任务的学科。 它可以用于回归（预测）、识别外观或声音以及其他形式的分析。 机器学习训练计算机去执行任务，而不必显式地编写程序来完成同样的任务。'

这也就是我们经常会看到一些大语言模型具备三种不同的模式，分别为精确模式、平衡模式和创造力模式（例如早期的Bing AI聊天），其功能的实现都是通过大模型的参数调整来完成的，而支持这些不同模式功能实现的最基础的两个参数就是presence_penalty参数和temperature参数。

#### 3.3 通过best_of获得更高质量输出文本

&emsp;&emsp;最后需要关注的参数则是best_of，通过设置该参数，能够让一个模型预测多组结果，并择优进行输出。例如这里设置best_of=5，则代表总共创建5个备选文本，再从中选取概率最大（得分最高）的一组进行输出：

In [158]:
response = openai.Completion.create(
           model="text-davinci-003",
           prompt="请问什么是机器学习",
           max_tokens=1000,
           presence_penalty=-1,
           temperature=1.2,
           best_of=5
           )

In [159]:
response["choices"][0]["text"].strip('？\n\n')

'机器学习是一种人工智能技术，它让计算机系统能够在没有被编程明确训练的情况下，通过给定的数据进行自我学习，从数据中学习，找出最佳匹配，并实现自动化预测和决策的技术。'

需要注意的是，由于大语言模型的输出结果本身无法复现，因此哪怕是设置了best_of，最终的输出结果也是无法复现的。

- best_of参数使用注意事项：      
(1).在大多数情况下，随着best_of值增加，最终的输出结果质量也会随之提升；      
(2).不过于此同时，best_of参数会消耗大量Token配额以及每分钟调用次数配额，同时也会消耗更多的资金；      
(3).如果说best_of是自动筛选最优结果，那么参数n则是可以支持手动筛选最佳结果，甚至有的时候可以同时使用这两个参数，以期获得更好的结果。当然，此时消耗的Token配额和调用次数配额也是呈现指数级增加的，例如best_of=2、n=2时，总共发出四次请求，最终返回两个结果：

In [160]:
response = openai.Completion.create(
           model="text-davinci-003",
           prompt="请问什么是机器学习",
           max_tokens=1000,
           presence_penalty=-1,
           temperature=1.2,
           n=2,
           best_of=2
           )

In [161]:
response["choices"][0]["text"].strip('？\n\n')

'机器学习是一门多面向的技术，它的基本思路是通过自学习来自动提取算法，使得计算机系统可以”记住“，”理解“和”操作“信息，它能够帮助计算机来自动完成任务，而不是复制现有的结果，机器学习算法允许计算机分析新的信息，并根据新的信息做出更有用的改变，其通过使得算法可以自动学习而不用被刻板的指令来实现新功能，改变创新而不需要重新编程。'

In [162]:
response["choices"][1]["text"].strip('？\n\n')

'机器学习是计算机程序自动地改进自身的一种学习算法，在给定的训练数据集上模型自动针对训练记录进行抽象学习并以此来进行后续对新任务数据的预测或处理。它和传统的Strong AI编程方法有着大相径庭的概念，后者是从程序中显式编码形式和细节来给计算机展示学习者应当如何做的。相反，机器式学习利用的是基于概率模型的学习方法，它以’学会’的方式解决问题，它用这种方法构建出的学习模型可以包含自动应用或者监督的方法模型学习。模型也可以用于强化。机器学习应用在深度学习算法以及机器视觉算法中得到实践广泛应用。'

#### 4.基于Completion.create函数的可调节对话风格的多轮对话机器人

&emsp;&emsp;在充分了解Completion.create函数功能之后，接下来我们尝试编写一个基于Completion.create函数的可以在Jupyter内部实现多轮对话的聊天机器人，同时通过预设不同的参数，在函数内部预设精确模式、平衡模式和创造力模式，以满足不同使用场景。函数定义如下：

In [1]:
def chat_now(model='text-davinci-003',mode='balance'):
    """
    基于Completion.create函数的多轮对话机器人
    
    :param model: 调用的大语言模型，默认为text-davinci-003
    :param mode: 聊天机器人预设模式，默认为平衡模式balance，可选precision（精确模式）和creativity（创造力模式）

    """
    # 提示想终止聊天时输入"quit"
    print("if you want to stop the conversation, please input 'quit'") 
    # 三种不同的模式及其对应的参数
    if mode == 'balance':
        temperature = 1
        presence_penalty = 0
    elif mode == 'precision':
        temperature = 0.8
        presence_penalty = 2
    elif mode == 'creativity':
        temperature = 1.2
        presence_penalty = -1     
    
    # 定义执行对话函数，方便后续反复调用
    def chat(prompt):
        try:
            # 不报错的情况下，返回Completion.create函数输出结果
            response = openai.Completion.create(
                       model = model,
                       prompt = prompt,
                       max_tokens = 1000,
                       temperature=temperature, 
                       presence_penalty=presence_penalty,
                       stop = [" Human:", " AI:"]
                       )

            answer = response["choices"][0]["text"].strip()
            return answer
        except Exception as exc:
            # 报错时返回"broken"
            return "broken"

    # 对话执行函数，首先准备空容器
    text = "" 
    turns = [] 
    # 执行多轮对话，即多次调用chat函数
    while True: 
        # 启动对话框
        question = input()
        # 首次开启对话框时提示请输入问题
        if len(question.strip()) == 0: 
            print("please input your question")
        # 当输入为'quit'时，停止多轮对话，即停止while循环
        elif question == "quit":  
            print("\nAI: See You Next Time!")
            break
        else:
            # 多轮对话时，将问题和此前对话结果都作为prompt输入
            prompt = text + "\nHuman: " + question
            result = chat(prompt)
            # 当一次请求失败时，再次发起请求
            while result == "broken": 
                print("please wait...")
                result = chat(prompt) 
            else:
                # 保留本次对话结果
                turns += [question] + [result]
                print(result)
            # 最多保留十次对话结果，超出次数则最开始的对话会被删除
            if len(turns)<=10:  
                text = " ".join(turns)
            else:
                text = " ".join(turns[-10:])

效果测试：

In [167]:
chat_now()

if you want to stop the conversation, please input 'quit'


 你好


Bot: 你好！很高兴见到你。


 请问什么是机器学习？


Bot: 机器学习是一种人工智能，它使用数据和算法来让计算机从观察和经验中学习，从而改善自身性能。


...

Bot: 机器学习的目标是使计算机可以根据内部数据建立模型，并能够根据新给定的输入不断调整和改进自身行为。它也帮助机器自动和持续的学习，使其在更复杂的任务上表现得更加灵活。


 我问你的第一个问题是什么问题呢？


Bot: 我的第一个问题是“请问什么是机器学习？”


 quit



AI: See You Next Time!
